<a href="https://colab.research.google.com/github/ykitaguchi77/GAN/blob/master/Scrape_Pinterest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Scrape images from Pinterest**

In [1]:
!git clone https://github.com/civiliangame/PinterestScraper.git
%cd PinterestScraper

Cloning into 'PinterestScraper'...
remote: Enumerating objects: 13, done.
remote: Total 13 (delta 0), reused 0 (delta 0), pack-reused 13
Unpacking objects: 100% (13/13), done.
/content/PinterestScraper


In [2]:
#Chromiumとseleniumをインストール
print("前処理を開始")
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

#!pip install -U google-colab
#!pip install -U datascience
#!pip install -U requests
#!pip install -U urllib3
!pip install selenium

#ライブラリをインポート
from selenium import webdriver
import time

#---------------------------------------------------------------------------------------
# 処理開始
#---------------------------------------------------------------------------------------
# ブラウザをheadlessモード実行
print("\nブラウザを設定")
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',options=options)
driver.implicitly_wait(10)

# # サイトにアクセス
# print("サイトにアクセス開始")
# driver.get("https://www.google.com/?hl=ja")
# time.sleep(3)

# # driver.find_elements_by_css_selector("xxx") 的な処理を自由に
# print("サイトのタイトル：", driver.title)

# print("\nお疲れさまです。\n処理が完了しました。")

前処理を開始
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:6 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:7 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:8 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:13 http://security.ubuntu.com/

#**Modules**

In [37]:
from bs4 import BeautifulSoup
import time
from selenium import webdriver
from selenium.webdriver.support import ui
from EnglishScraper import ScrapingEssentials
import threading
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import os
import shutil


def page_is_loaded(driver):
    return driver.find_element(By.TAG_NAME, "body") != None

def login(driver, username, password):
    if driver.current_url != "https://www.pinterest.com/login/?referrer=home_page":
        driver.get("https://www.pinterest.com/login/?referrer=home_page")
    wait = ui.WebDriverWait(driver, 10)
    wait.until(page_is_loaded)
    email = driver.find_element(By.XPATH, "//input[@type='email']")
    password = driver.find_element(By.XPATH,"//input[@type='password']")
    email.send_keys("sxh779@case.edu")
    password.send_keys("multithreading")
    # driver.find_element_by_xpath("//div[@data-reactid='30']").click()
    password.submit()
    time.sleep(3)
    print("Teleport Successful!")

# Finds the detailed product page of each "pin" for pinterest
def download_pages(driver, valid_urls):
    list_counter = 0

    # Pinterest happens to change its HTML every once in a while to prevent botting.

    # This should account for all the differences
    # soup = BeautifulSoup(driver.page_source, "lxml")
    # for pinWrapper in soup.find_all("div", {"class": "pinWrapper"}):
    #     class_name = pinWrapper.get("class")
    #     print(class_name)
    #     if "_o" in class_name[0]:
    #         print(class_name)
    #         break
    #
    # #Finds the tags of the HTML and adjusts it
    # name = " ".join(class_name)
    # print(name)

    # Does this until you have 10000 items or the program has gone on for long enough, meaning that it reached the end of results
    beginning = time.time()
    end = time.time()
    while list_counter < 10000 and beginning - end < 30:
        beginning = time.time()
        # ----------------------------------EDIT THE CODE BELOW------------------------------#
        # Locate all the urls of the detailed pins
        soup = BeautifulSoup(driver.page_source, "html.parser")
        # for c in soup.find_all("div", {"class": name}):
        for pinLink in soup.find_all("div", {"class": "pinWrapper"}):
            for a in pinLink.find_all("a"):
                print(pinLink)
                url = ("https://pinterest.com" + str(a.get("href")))
                print(url)
                # Checks and makes sure that the pin isn't there already and that random urls are not invited
                if len(url) < 60 and url not in valid_urls and "A" not in url:
                    # ---------------------------------EDIT THE CODE ABOVE-------------------------------#
                    valid_urls.append(url)
                    print("found the detailed page of: " + str(list_counter))
                    list_counter += 1
                    end = time.time()
                time.sleep(.15)
                # Scroll down now
        driver.execute_script("window.scrollBy(0,300)")
    return

# Downloads the image files from the img urls
def get_pic(valid_urls, driver):
    print("hey")
    get_pic_counter = 0
    time.sleep(5)
    while (get_pic_counter < len(valid_urls)):
        print(0)
        # Now, we can just type in the URL and pinterest will not block us
        for urls in valid_urls:
            driver.get(urls)
            # Wait until the page is loaded
            if driver.current_url == urls:
                wait = ui.WebDriverWait(driver, 10)
                wait.until(page_is_loaded)
                loaded = True
            print(1)
            # -----------------------------------EDIT THE CODE BELOW IF PINTEREST CHANGES---------------------------#
            # Extract the image url
            soup = BeautifulSoup(driver.page_source, "html.parser")
            print(2)
            for mainContainer in soup.find_all("div", {"class": "mainContainer"}):
                print(3)
                for closeupContainer in mainContainer.find_all("div", {"class": "closeupContainer"}):
                    print(4)
                    # for heightContainer in closeupContainer.find_all("div", {"class": "FlashlightEnabledImage Module"}):
                    print(5)
                    for img in closeupContainer.find_all("img"):
                        print(6)
                        print("hello")
                        img_link = img.get("src")
                        if "564" in img_link:
                            print("found the img url of: " + str(get_pic_counter))
                            get_pic_counter += 1
                            t.download_image(img_link)
                            break

def get_url_list(keyword, img_num, driver):
    
    #ページを開く
    driver.get("https://pinterest.com/search/pins/?q=" + str(keyword) + "&rs=typed&term_meta[]=" + str(
              keyword) + "%7Ctyped")
    #ページ読み込み待ち
    wait = ui.WebDriverWait(driver, 10)
    wait.until(page_is_loaded)
    
    all_pin_data = []
    while 1:
        # scroll
        driver1.execute_script("window.scrollBy(0,10000)")
        time.sleep(3)
        driver1.execute_script("window.scrollBy(0,10000)")
        time.sleep(3)

        # get the html now
        page_source = driver.page_source
        page = BeautifulSoup(page_source, 'html.parser')

        # 'GrowthUnauthPinImage' is the class of all the pins
        # Here we take divs which have a with href as pin number
        pin_data = page.find_all('div', "Yl- MIw Hb7")
        all_pin_data.extend(pin_data)
        temp = all_pin_data
        all_pin_data = list(set(all_pin_data)) #リスト内の重複を削除

        print("duplicate: {}".format(len(temp) - len(all_pin_data)))
        print("total: {}".format(len(all_pin_data)))
        print("")

        #pin_dataの数がimg_numを超えたら終了
        if len(all_pin_data) > img_num:
            break

    # アドレスのリストを追加
    url_list = []
    for i in range(len(all_pin_data)):
        url_list.append('https://www.pinterest.com' +
                    all_pin_data[i].find('a')['href'])
        print('https://www.pinterest.com' +
                    all_pin_data[i].find('a')['href']) 
    return url_list



    def get_url_img(url, driver):
    
        #ページを開く
        driver.get(url)
        #ページ読み込み待ち
        wait = ui.WebDriverWait(driver, 10)
        wait.until(page_is_loaded)
        
        page_source = driver.page_source
        page = BeautifulSoup(page_source, 'html.parser')

        

    while 1:
        # scroll
        driver1.execute_script("window.scrollBy(0,10000)")
        time.sleep(3)
        driver1.execute_script("window.scrollBy(0,10000)")
        time.sleep(3)

        # get the html now
        

        # 'GrowthUnauthPinImage' is the class of all the pins
        # Here we take divs which have a with href as pin number
        pin_data = page.find_all('div', "Yl- MIw Hb7")
        all_pin_data.extend(pin_data)
        temp = all_pin_data
        all_pin_data = list(set(all_pin_data)) #リスト内の重複を削除

        print("duplicate: {}".format(len(temp) - len(all_pin_data)))
        print("total: {}".format(len(all_pin_data)))
        print("")

        #pin_dataの数がimg_numを超えたら終了
        if len(all_pin_data) > img_num:
            break

    # アドレスのリストを追加
    url_list = []
    for i in range(len(all_pin_data)):
        url_list.append('https://www.pinterest.com' +
                    all_pin_data[i].find('a')['href'])
        print('https://www.pinterest.com' +
                    all_pin_data[i].find('a')['href']) 
    return url_list
        


            # ---------------------------------EDIT THE CODE ABOVE IF PINTEREST CHANGES-----------------------------#


In [ ]:
t = ScrapingEssentials("Pinterest")
keyword_list = t.english_pickle()
print(keyword_list)

#ChromeDriverをheadlessモードにする
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')


driver1 = webdriver.Chrome('chromedriver',options=options)
driver2 = webdriver.Chrome('chromedriver',options=options)
driver1.implicitly_wait(10) # 秒

driver1.get("https://www.pinterest.com/login/?referrer=home_page")
driver2.get("https://www.pinterest.com/login/?referrer=home_page")

login(driver1, "e-mail", "password")
login(driver2, "e-mail", "password")

# loaded = False
# while loaded == False:
#     if driver1.current_url != "https://www.pinterest.com/login/?referrer=home_page":
#         loaded = True

#カレントページのURLを取得して表示
cur_url = driver1.current_url
print("current_url: {}".format(cur_url))

#
for keyword in keyword_list:
    print("start")
    print(keyword)

    url_list = get_url_list(keyword, 50,driver1)

    
print("done")

In [28]:
get_url_list("girls", 50, driver1)

duplicate: 0
total: 17

duplicate: 0
total: 34

duplicate: 0
total: 49

duplicate: 0
total: 63



#**動きの確認**

In [8]:
print(keyword)

Kia


In [9]:
#アドレスが開くかどうかを確認
driver1.get("https://pinterest.com/search/pins/?q=" + str(keyword) + "&rs=typed&term_meta[]=" + str(
            keyword) + "%7Ctyped")


In [10]:
#カレントページのURLを取得して表示
cur_url = driver1.current_url
print("current_url: {}".format(cur_url))

current_url: https://www.pinterest.com/search/pins/?q=Kia&rs=typed&term_meta[]=Kia%7Ctyped


In [11]:
driver1.execute_script("window.scrollBy(0,10000)")
time.sleep(3)
driver1.execute_script("window.scrollBy(0,10000)")
time.sleep(3)

In [ ]:
soup = BeautifulSoup(driver1.page_source, "html.parser")
print(soup)

In [19]:
import pandas as pd


def get_url_list(keyword, driver):
    
    #ページを開く
    driver.get("https://pinterest.com/search/pins/?q=" + str(keyword) + "&rs=typed&term_meta[]=" + str(
              keyword) + "%7Ctyped")
    #ページ読み込み待ち
    wait = ui.WebDriverWait(driver, 10)
    wait.until(page_is_loaded)
    
    all_pin_data = []
    while 1:
        # scroll
        driver1.execute_script("window.scrollBy(0,10000)")
        time.sleep(3)
        driver1.execute_script("window.scrollBy(0,10000)")
        time.sleep(3)

        # get the html now
        page_source = driver.page_source
        page = BeautifulSoup(page_source, 'html.parser')

        # 'GrowthUnauthPinImage' is the class of all the pins
        # Here we take divs which have a with href as pin number
        pin_data = page.find_all('div', "Yl- MIw Hb7")
        all_pin_data.extend(pin_data)
        temp = all_pin_data
        all_pin_data = list(set(all_pin_data)) #リスト内の重複を削除

        print("duplicate: {}".format(len(temp) - len(all_pin_data)))
        print("total: {}".format(len(all_pin_data)))
        print("")

        if len(all_pin_data) > 50:
            break

    # アドレスのリストを追加
    hrefs = []
    for i in range(len(all_pin_data)):
        hrefs.append('https://www.pinterest.com' +
                    all_pin_data[i].find('a')['href'] + 'visual-search/')

# save to CSV
df = pd.DataFrame({'PID_URLS': hrefs[:50]})
df.to_csv("pintrest.csv", index=False)

duplicate: 0
total: 21

duplicate: 0
total: 44

duplicate: 0
total: 68



In [40]:
url = "https://www.pinterest.com/pin/516295544771307338/"

driver2.get(url)

In [41]:
  page_source = driver2.page_source
  page = BeautifulSoup(page_source, 'html.parser')

In [43]:
pin_data = page.find_all('div', "Yl- MIw Hb7")
print(pin_data)

[<div class="Yl- MIw Hb7" data-grid-item="true" role="listitem" style="top: 0px; left: 0px; transform: translateX(8px) translateY(0px); width: 236px; height: 274px;"><div aria-disabled="false" class="CCY czT eEj iyn DI9 BG7" role="button" tabindex="0"><div class="zI7 iyn Hsu"><div class="zI7 iyn Hsu zmN" data-test-id="pin" data-test-pin-id="303570831128517381"><div class="lnZ wsz zI7 iyn Hsu zmN" data-test-id="pin-with-alt-text"><div class="zI7 iyn Hsu" data-test-id="deeplink-wrapper"><a class="Wk9 CCY czT ho- kVc xQ4 iyn" href="/pin/303570831128517381/" rel=""><div class="PinCard__imageWrapper" style="position: relative;"><div class="XiG ho- sLG zI7 iyn Hsu" data-test-id="pincard-image-with-link" style="will-change: transform; text-decoration: none;"><div class="XiG zI7 iyn Hsu" data-test-id="pin-visual-wrapper"><div class="Pj7 sLG XiG ho- m1e"><div class="XiG zI7 iyn Hsu" style="background-color: rgb(121, 128, 129); padding-bottom: 66.1017%;"><img alt="Post Your Pictures Here! - Page

In [44]:
keyword = "赤い人"

print("https://pinterest.com/search/pins/?q=" + str(keyword) + "&rs=typed&term_meta[]=" + str(
              keyword) + "%7Ctyped")

https://pinterest.com/search/pins/?q=赤い人&rs=typed&term_meta[]=赤い人%7Ctyped
